In [1]:
import pandas as pd
import numpy as np
from pmdarima.arima import AutoARIMA
import plotly.express as px
import plotly.graph_objects as go
from tqdm.notebook import tqdm
from sklearn.metrics import mean_squared_error
from datetime import date as dt, timedelta
import yfinance as yf
import pymysql
import warnings
warnings.filterwarnings('ignore')
import datetime as dt
import pandas_datareader as web
from dateutil.relativedelta import *
import yfinance as yf
from yahoofinancials import YahooFinancials
from sqlalchemy import create_engine
import mysql.connector

In [2]:
today_date = dt.datetime.today().strftime('%m-%d-%Y')

In [3]:
def publish_one(final_cal_df, table_name):
    conn=pymysql.connect(host='*******',
                         port=int(*****),
                         user='*******',
                         passwd='*******',
                         db='Options')

    my_conn = create_engine('mysql+mysqlconnector://*****:*******@******:******/options', echo=False)
    final_cal_df.to_sql(name=table_name, con = my_conn, if_exists='append', index=False)

In [4]:
def unsual_option_actv():
    
    unusual_stock_options_activity = pd.read_csv(f'unusual-stock-options-activity-{today_date}.csv')
    unusual_stock_options_activity = unusual_stock_options_activity.iloc[:-1,:]
    unusual_stock_options_activity.rename(columns={'Exp Date': 'Exp_Date', 'Open Int': 'Open_Int'}, inplace=True)
    unusual_stock_options_activity.IV = unusual_stock_options_activity.IV.str.replace(r',', "")
    unusual_stock_options_activity.IV = unusual_stock_options_activity.IV.str.replace(r'%', "").astype('float')
    
    publish_one(unusual_stock_options_activity, 'unusual_stock_options_activity')
    
    print('unusual_stock_options_activity uploaded successfully')
    print('x'*100)
    
#     return unusual_stock_options_activity

# unsual_option_actv()

In [5]:
def most_active_optns():

    most_active_options = pd.read_csv(f'most-active-stock-options-{today_date}.csv')
    most_active_options = most_active_options.iloc[:-1,:]

    most_active_options.rename(columns={'IV Rank':'IV_Rank', 
                                              'Options Vol':'Options_Vol', 
                                              '% Put':'%_Put', 
                                              '% Call':'%_Call',
                                              'Put/Call Vol':'Put/Call_Vol'}, inplace=True)

    cols = ['%Chg', 'IV_Rank', '%_Put', '%_Call']

    for col in cols:
        most_active_options[col] = most_active_options[col].str.replace(r'%', "").astype('float')

    publish_one(most_active_options, 'most_active_stock_options')
    
    print('most_active_stock_options uploaded successfully')
    print('x'*100)
    
#     return most_active_options
    
# most_active_optns()

In [6]:
def unusual_stock_opt_vol():

    unusual_stock_vol = pd.read_csv(f'unusual-stock-options-volume-{today_date}.csv')
    unusual_stock_vol = unusual_stock_vol.iloc[:-1,:]
    
    unusual_stock_vol.rename(columns={'Options Vol':'Options_Vol',
                                 'Total OI':'Total_OI',
                                 'IV Rank': 'IV_Rank',
                                 '1M Vol %Chg':'1M_Vol_%Chg',
                                 'Call Volume':'Call_Volume',
                                 'Put Volume':'Put_Volume',
                                 'Put/Call Vol':'Put/Call_Vol'}, inplace=True)
    
    
    cols = ['%Chg', 'IV_Rank']
    for col in cols:
            unusual_stock_vol[col] = unusual_stock_vol[col].str.replace(r'%', "").astype('float')

    unusual_stock_vol['1M_Vol_%Chg'] = unusual_stock_vol['1M_Vol_%Chg'].str.replace(r'%', "")
    unusual_stock_vol['1M_Vol_%Chg'] = unusual_stock_vol['1M_Vol_%Chg'].str.replace(r',', "").astype('float')
    unusual_stock_vol['Time'] = pd.to_datetime(today_date)
    
    publish_one(unusual_stock_vol, 'unusual_stock_options_volume')
    
    print('unusual_stock_options_volume uploaded successfully')
    print('x'*100)
    
#     return unusual_stock_vol

# unusual_stock_opt_vol()

In [7]:
def stock_opt_inc_dec_iv():    
    
    stocks_increased_iv = pd.read_csv(f'stocks-increase-percent-change-in-volatility-{today_date}.csv')
    stocks_decreased_iv = pd.read_csv(f'stocks-decrease-percent-change-in-volatility-{today_date}.csv')

    stocks_increased_iv = stocks_increased_iv.iloc[:-1,:]
    stocks_decreased_iv = stocks_decreased_iv.iloc[:-1,:]

    stocks_increased_iv.rename(columns={'Exp Date':'Exp_Date',
                                       'Open Int':'Open_Int',
                                       'IV %Chg':'IV_%Chg'}, inplace=True)
    stocks_decreased_iv.rename(columns={'Exp Date':'Exp_Date',
                                       'Open Int':'Open_Int',
                                       'IV %Chg':'IV_%Chg'}, inplace=True)


    cols = ['IV_%Chg', 'IV']
    for col in cols:
            stocks_increased_iv[col] = stocks_increased_iv[col].str.replace(r'%', "").astype('float')
            stocks_decreased_iv[col] = stocks_decreased_iv[col].str.replace(r'%', "").astype('float')

    publish_one(stocks_increased_iv, 'stocks_increase_iv')
    publish_one(stocks_decreased_iv, 'stocks_decrease_iv')
    
    print('stocks_increase_iv & stocks_decrease_iv  uploaded successfully')
    print('x'*100)

#     return stocks_increased_oi, stocks_decreased_oi

# stock_opt_inc_dec_iv()

In [8]:
def stock_opt_volume_leaders():    
    
    stock_vol_leaders = pd.read_csv(f'stock-options-volume-leaders-{today_date}.csv')
    stock_vol_leaders = stock_vol_leaders.iloc[:-1,:]

    stock_vol_leaders.rename(columns={'Exp Date':'Exp_Date',
                                     'Open Int':'Open_Int'}, inplace=True)
    
    stock_vol_leaders['IV'] = stock_vol_leaders['IV'].str.replace(r',', "")
    stock_vol_leaders['IV'] = stock_vol_leaders['IV'].str.replace(r'%', "").astype('float')

    publish_one(stock_vol_leaders, 'stock_options_volume_leaders')
    
    print('stock_options_volume_leaders  uploaded successfully')
    print('x'*100)

#     return stock_vol_leaders

# stock_opt_volume_leaders()

In [9]:
def stock_change_in_oi():    
    
    stock_increased_oi = pd.read_csv(f'stocks-increase-change-in-open-interest-{today_date}.csv')
    stock_decreased_oi = pd.read_csv(f'stocks-decrease-change-in-open-interest-{today_date}.csv')

    stock_increased_oi = stock_increased_oi.iloc[:-1,:]
    stock_decreased_oi = stock_decreased_oi.iloc[:-1,:]
    
    stock_increased_oi  = stock_increased_oi[stock_increased_oi['OI Chg'] != 'unch']
    stock_decreased_oi  = stock_decreased_oi[stock_decreased_oi['OI Chg'] != 'unch']

    stock_increased_oi.rename(columns={'Exp Date':'Exp_Date',
                                      'Open Int':'Open_Int',
                                      'OI Chg':'OI_Chg'}, inplace=True)
    stock_decreased_oi.rename(columns={'Exp Date':'Exp_Date',
                                      'Open Int':'Open_Int',
                                      'OI Chg':'OI_Chg'}, inplace=True)

    stock_increased_oi['OI_Chg'] =  stock_increased_oi['OI_Chg'].str.replace(r',', "").astype('float')
    stock_increased_oi['IV'] = stock_increased_oi['IV'].str.replace(r',', "")
    stock_increased_oi['IV'] = stock_increased_oi['IV'].str.replace(r'%', "").astype('float')

    stock_decreased_oi['OI_Chg'] =  stock_decreased_oi['OI_Chg'].str.replace(r',', "").astype('float')
    stock_decreased_oi['IV'] = stock_decreased_oi['IV'].str.replace(r',', "")
    stock_decreased_oi['IV'] = stock_decreased_oi['IV'].str.replace(r'%', "").astype('float')

    publish_one(stock_increased_oi, 'stocks_increase_oi')
    publish_one(stock_decreased_oi, 'stocks_decrease_oi')
    
    print('stocks_increase_oi & stocks_decrease_oi uploaded successfully')
    print('x'*100)

#     return stock_increased_oi, stock_decreased_oi

# stock_change_in_oi()

In [10]:
def highest_lowest_iv():    
    
    highest_iv = pd.read_csv(f'stocks-highest-implied-volatility-{today_date}.csv')
    lowest_iv = pd.read_csv(f'stocks-lowest-implied-volatility-{today_date}.csv')

    highest_iv = highest_iv.iloc[:-1,:]
    lowest_iv = lowest_iv.iloc[:-1,:]

    highest_iv.rename(columns={'Exp Date':'Exp_Date',
                              'Open Int':'Open_Int'}, inplace=True)
    lowest_iv.rename(columns={'Exp Date':'Exp_Date',
                              'Open Int':'Open_Int'}, inplace=True)

    highest_iv['IV'] = highest_iv['IV'].str.replace(r'%', "").astype('float')
    lowest_iv['IV'] = lowest_iv['IV'].str.replace(r'%', "").astype('float')

    publish_one(highest_iv, 'stocks_highest_iv')
    publish_one(lowest_iv, 'stocks_lowest_iv')
    
    print('stocks_highest_iv & stocks_lowest_iv uploaded successfully')
    print('x'*100)

#     return highest_iv, lowest_iv

# highest_lowest_iv()

In [11]:
def main():
    
    unsual_option_actv()
    most_active_optns()
    unusual_stock_opt_vol()
    stock_opt_inc_dec_iv()
    stock_opt_volume_leaders()
    stock_change_in_oi()
    highest_lowest_iv()

In [12]:
main()

unusual_stock_options_activity uploaded successfully
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
most_active_stock_options uploaded successfully
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
unusual_stock_options_volume uploaded successfully
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
stocks_increase_iv & stocks_decrease_iv  uploaded successfully
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
stock_options_volume_leaders  uploaded successfully
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
stocks_increase_oi & stocks_decrease_oi uploaded successfully
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
stocks_highest_iv & stocks_lowest_iv uploaded successfully
xxxxxx